# Model

The puspose of this notebook is to create a model and all the features that can help to improve the accuracy of such model.

It is been decided to use a Random Forest due to the good results it gives

We will follow the following points:

1. Create the dataframe
2. Times feature engineering
3. Lag Variables
4. Weather feature engineering
5. The function for all the feature engineering
6. Model
    1. First Model
    2. RandomizedSearchCV Model
    3. Test

*NOTE: Kindly note that this notebook has been "cleaned up" and all the code has been organized for your easy understanding.*

## 1. Create the dataframe

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import os
pd.set_option('display.max_columns', None)
from datetime import datetime

In [2]:
#Get the preprocessig_v2.py data path
cwd = os.getcwd()
cwd_list = cwd.split(os.sep)
preprocessing_path = os.path.join(os.sep.join(cwd_list[:-1]), '02-EDA')
preprocessing_path

'C:\\Users\\Aeroengy\\Desktop\\TFM\\02-EDA'

In [3]:
%cd $preprocessing_path
import preprocessing_v2 as pre
%cd $cwd

C:\Users\Aeroengy\Desktop\TFM\02-EDA
C:\Users\Aeroengy\Desktop\TFM\03-MODEL


In [4]:
#Define data path WINDOWS
data_path = os.path.join(os.sep.join(cwd_list[:-1]), 'DATA')
data_path

'C:\\Users\\Aeroengy\\Desktop\\TFM\\DATA'

In [5]:
#Define data path LINUX
# data_path = '/home/gonzalo/Data/TFM/DATA/'
# data_path

In [6]:
df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=True )

0%[--10%------------------]100%
Bicimad_Estacions_201808.json added to DataSet
0%[----20%----------------]100%
0%[------30%--------------]100%
Bicimad_Estacions_201809.json added to DataSet
0%[--------40%------------]100%
0%[----------50%----------]100%
Bicimad_Estacions_201810.json added to DataSet
0%[------------60%--------]100%
Bicimad_Stations_201811.json added to DataSet
0%[--------------70%------]100%
0%[----------------80%----]100%
Bicimad_Stations_201812.json added to DataSet
0%[------------------90%--]100%
Bicimad_Stations_201901.json added to DataSet


In [7]:
df_total_bases = pre.total_bases_dataset(data_path)

In [8]:
df_occupation_rate = pre.make_df_occupation_rate(df_dock_bikes, df_total_bases)

In [9]:
df_total_bases.describe()

,28013_total_bases,28015_total_bases,28004_total_bases,28010_total_bases,28005_total_bases,28008_total_bases,28014_total_bases,28012_total_bases,28009_total_bases,28007_total_bases,28001_total_bases,28006_total_bases,28020_total_bases,28002_total_bases,28045_total_bases,28003_total_bases,28046_total_bases,28036_total_bases,28016_total_bases
count,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000,4422.000000
mean,215.472863,127.050204,335.457033,114.683853,242.411126,186.023066,179.636590,261.184080,333.454093,92.608096,201.455902,184.428087,158.227725,109.790366,116.201945,134.442108,81.158526,70.592040,23.499774
std,40.228308,5.691990,9.059647,4.771789,12.986139,4.272012,6.647029,12.303283,5.656858,2.471179,7.964166,6.042274,5.380389,3.355734,2.597158,4.500237,8.149972,2.757976,0.733331
min,87.000000,97.000000,301.000000,86.000000,197.000000,143.000000,138.000000,210.000000,299.000000,70.000000,144.000000,152.000000,111.000000,63.000000,89.000000,108.000000,0.000000,46.000000,20.000000
25%,221.000000,125.000000,329.000000,113.000000,235.000000,184.000000,177.000000,254.000000,331.000000,91.000000,201.000000,183.000000,157.000000,109.000000,115.000000,133.000000,81.000000,70.000000,23.000000
50%,235.000000,128.000000,337.000000,116.000000,246.000000,187.000000,181.000000,263.000000,334.000000,93.000000,203.000000,186.000000,159.000000,110.000000,117.000000,135.000000,82.000000,71.000000,24.000000
75%,238.000000,131.000000,343.000000,117.000000,252.000000,189.000000,184.000000,270.000000,337.000000,94.000000,205.000000,188.000000,161.000000,112.000000,118.000000,137.000000,83.000000,72.000000,24.000000
max,245.000000,135.000000,356.000000,120.000000,261.000000,194.000000,189.000000,297.000000,344.000000,96.000000,210.000000,192.000000,165.000000,114.000000,120.000000,140.000000,84.000000,72.000000,24.000000


In [10]:
df_occupation_rate.head()

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate
Date,,,,,,,,,,,,,,,,,,,
2018-08-01,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932
2018-08-02,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505
2018-08-03,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307
2018-08-04,0.365708,0.366367,0.181165,0.393908,0.389521,0.472140,0.501181,0.456476,0.515255,0.554839,0.512104,0.445342,0.493756,0.476997,0.544531,0.520857,0.519066,0.555746,0.491611
2018-08-05,0.352376,0.460019,0.368582,0.520932,0.378999,0.521230,0.407549,0.415905,0.442334,0.512357,0.470336,0.532366,0.468555,0.500566,0.485523,0.548259,0.509181,0.499412,0.452080


In [56]:
df_occupation_rate.shape

(185, 19)

#### Let's divide the dataset into train, validaton an the other one for test.

In [11]:
train = df_occupation_rate.iloc[:-2]
train.tail(1)

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate
Date,,,,,,,,,,,,,,,,,,,
2019-01-30,0.48614,0.512156,0.556675,0.407846,0.564081,0.463787,0.35214,0.420886,0.532424,0.526549,0.385246,0.402423,0.458475,0.437238,0.438652,0.613636,0.431234,0.553488,0.576655


In [12]:
validation = df_occupation_rate[-2:-1]
validation

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate
Date,,,,,,,,,,,,,,,,,,,
2019-01-31,0.53734,0.454545,0.506321,0.418848,0.471588,0.431507,0.356584,0.48739,0.547666,0.411472,0.436453,0.521556,0.487042,0.429854,0.450518,0.554733,0.537722,0.610229,0.521336


In [13]:
test = df_occupation_rate[-1:]
test

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate
Date,,,,,,,,,,,,,,,,,,,
2019-02-01,0.487179,0.576,0.473837,0.369369,0.602362,0.407609,0.325843,0.689781,0.561765,0.677778,0.131068,0.438503,0.525974,0.306306,0.422414,0.597015,0.578313,0.614286,0.714286


#### Kindly note that the split is just informative, we will split the dataset before train the model. In the following engineering we will use the train with validations because it is needed the day to predict to get some features

## 2. Times feature engineering

The procedure we are going to follow is the following:

- Make the feature day_month, since it may be important to know if we are at the beginning or end of the month.
- Make the feature back_days, which will be the number of days between the day to predict and the sample.
- Maje the feature weekday which will be divided in two, since in order to keep the information that the sunday is next to monday, we will divide by coordinates where the days of the week will be aroun a circle as a clock.

In [14]:
train = df_occupation_rate.iloc[:-1]
train.head()

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate
Date,,,,,,,,,,,,,,,,,,,
2018-08-01,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932
2018-08-02,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505
2018-08-03,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307
2018-08-04,0.365708,0.366367,0.181165,0.393908,0.389521,0.472140,0.501181,0.456476,0.515255,0.554839,0.512104,0.445342,0.493756,0.476997,0.544531,0.520857,0.519066,0.555746,0.491611
2018-08-05,0.352376,0.460019,0.368582,0.520932,0.378999,0.521230,0.407549,0.415905,0.442334,0.512357,0.470336,0.532366,0.468555,0.500566,0.485523,0.548259,0.509181,0.499412,0.452080


In [15]:
 train.index[2].day

3

In [16]:
train['day_month'] = train.index.day
train.head()

C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month
Date,,,,,,,,,,,,,,,,,,,,
2018-08-01,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932,1
2018-08-02,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505,2
2018-08-03,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307,3
2018-08-04,0.365708,0.366367,0.181165,0.393908,0.389521,0.472140,0.501181,0.456476,0.515255,0.554839,0.512104,0.445342,0.493756,0.476997,0.544531,0.520857,0.519066,0.555746,0.491611,4
2018-08-05,0.352376,0.460019,0.368582,0.520932,0.378999,0.521230,0.407549,0.415905,0.442334,0.512357,0.470336,0.532366,0.468555,0.500566,0.485523,0.548259,0.509181,0.499412,0.452080,5


In [17]:
train.index[-1]

Timestamp('2019-01-31 00:00:00', freq='D')

In [18]:
date_to_predict =  train.index[-1] + 1

C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  """Entry point for launching an IPython kernel.


In [19]:
diff_dates = date_to_predict - train.index

In [20]:
train['back_days'] = diff_dates.days
train.tail(5)

C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-27,0.299477,0.385396,0.162701,0.203048,0.494028,0.653363,0.407425,0.550138,0.632122,0.667118,0.347915,0.643278,0.526451,0.603318,0.660714,0.490738,0.710355,0.537703,0.715044,27,5
2019-01-28,0.368841,0.332530,0.215184,0.382485,0.444240,0.560981,0.454824,0.507675,0.597810,0.568873,0.498269,0.551532,0.519191,0.628977,0.582752,0.668240,0.658488,0.633393,0.870567,28,4
2019-01-29,0.448655,0.436226,0.428039,0.453835,0.521565,0.520196,0.451750,0.458112,0.482685,0.652618,0.372763,0.426589,0.468322,0.489508,0.538020,0.620786,0.532898,0.645218,0.799283,29,3
2019-01-30,0.486140,0.512156,0.556675,0.407846,0.564081,0.463787,0.352140,0.420886,0.532424,0.526549,0.385246,0.402423,0.458475,0.437238,0.438652,0.613636,0.431234,0.553488,0.576655,30,2
2019-01-31,0.537340,0.454545,0.506321,0.418848,0.471588,0.431507,0.356584,0.487390,0.547666,0.411472,0.436453,0.521556,0.487042,0.429854,0.450518,0.554733,0.537722,0.610229,0.521336,31,1


In [21]:
train.index[2].weekday()

4

In [22]:
days_of_week = 7

train['sin_weekday'] = np.sin(2*np.pi*train.index.weekday/days_of_week)
train['cos_weekday'] = np.cos(2*np.pi*train.index.weekday/days_of_week)

train.head(14)

C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days,sin_weekday,cos_weekday
Date,,,,,,,,,,,,,,,,,,,,,,,
2018-08-01,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932,1,184,0.974928,-0.222521
2018-08-02,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505,2,183,0.433884,-0.900969
2018-08-03,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307,3,182,-0.433884,-0.900969
2018-08-04,0.365708,0.366367,0.181165,0.393908,0.389521,0.472140,0.501181,0.456476,0.515255,0.554839,0.512104,0.445342,0.493756,0.476997,0.544531,0.520857,0.519066,0.555746,0.491611,4,181,-0.974928,-0.222521
2018-08-05,0.352376,0.460019,0.368582,0.520932,0.378999,0.521230,0.407549,0.415905,0.442334,0.512357,0.470336,0.532366,0.468555,0.500566,0.485523,0.548259,0.509181,0.499412,0.452080,5,180,-0.781831,0.623490
2018-08-06,0.511089,0.479091,0.371330,0.407967,0.399603,0.452786,0.417554,0.438234,0.439910,0.432203,0.498646,0.460795,0.422601,0.496408,0.465886,0.460161,0.515429,0.515355,0.447684,6,179,0.000000,1.000000
2018-08-07,0.508705,0.458741,0.395698,0.385221,0.445691,0.535492,0.421878,0.490142,0.452694,0.478778,0.427686,0.459800,0.365116,0.439323,0.485547,0.471471,0.509718,0.483492,0.240964,7,178,0.781831,0.623490
2018-08-08,0.409048,0.463698,0.414617,0.412958,0.547779,0.511915,0.397177,0.625920,0.456338,0.458944,0.379381,0.438522,0.422281,0.387617,0.500182,0.481905,0.411286,0.465956,0.385714,8,177,0.974928,-0.222521
2018-08-09,0.523793,0.381074,0.373979,0.447518,0.595662,0.480821,0.406510,0.467355,0.487999,0.552320,0.406807,0.412836,0.448106,0.396284,0.460503,0.466543,0.448503,0.561877,0.575652,9,176,0.433884,-0.900969


In [23]:
def make_time_features(df):
    date_to_predict =  df.index[-1] + 1
    diff_dates = date_to_predict - df.index
    df['day_month'] = df.index.day
    df['back_days'] = diff_dates.days
    df['sin_weekday'] = np.sin(2*np.pi*df.index.weekday/7)
    df['cos_weekday'] = np.cos(2*np.pi*df.index.weekday/7)
    return df    

## 3. Lag Variables

In this case we will create a variable of how the occuppation rate was yesterday, 7 days before, and 30 days before.

In [24]:
def lag_function(df, column):
    df[column + '_' + 'lag' + '_' + '1'] = df[column].shift(1)
    df[column + '_' + 'lag' + '_' + '7'] = df[column].shift(7)
    df[column + '_' + 'lag' + '_' + '30'] = df[column].shift(30)
    return df

In [25]:
columns = [x for x in train if 'OccupationRate' in x]
columns

['28013_OccupationRate',
 '28015_OccupationRate',
 '28004_OccupationRate',
 '28010_OccupationRate',
 '28005_OccupationRate',
 '28008_OccupationRate',
 '28014_OccupationRate',
 '28012_OccupationRate',
 '28009_OccupationRate',
 '28007_OccupationRate',
 '28001_OccupationRate',
 '28006_OccupationRate',
 '28020_OccupationRate',
 '28002_OccupationRate',
 '28045_OccupationRate',
 '28003_OccupationRate',
 '28046_OccupationRate',
 '28036_OccupationRate',
 '28016_OccupationRate']

In [26]:
train.head()

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days,sin_weekday,cos_weekday
Date,,,,,,,,,,,,,,,,,,,,,,,
2018-08-01,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932,1,184,0.974928,-0.222521
2018-08-02,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505,2,183,0.433884,-0.900969
2018-08-03,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307,3,182,-0.433884,-0.900969
2018-08-04,0.365708,0.366367,0.181165,0.393908,0.389521,0.472140,0.501181,0.456476,0.515255,0.554839,0.512104,0.445342,0.493756,0.476997,0.544531,0.520857,0.519066,0.555746,0.491611,4,181,-0.974928,-0.222521
2018-08-05,0.352376,0.460019,0.368582,0.520932,0.378999,0.521230,0.407549,0.415905,0.442334,0.512357,0.470336,0.532366,0.468555,0.500566,0.485523,0.548259,0.509181,0.499412,0.452080,5,180,-0.781831,0.623490


In [27]:
for column in columns:
    train = lag_function(train, column)

C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [28]:
train.tail()

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days,sin_weekday,cos_weekday,28013_OccupationRate_lag_1,28013_OccupationRate_lag_7,28013_OccupationRate_lag_30,28015_OccupationRate_lag_1,28015_OccupationRate_lag_7,28015_OccupationRate_lag_30,28004_OccupationRate_lag_1,28004_OccupationRate_lag_7,28004_OccupationRate_lag_30,28010_OccupationRate_lag_1,28010_OccupationRate_lag_7,28010_OccupationRate_lag_30,28005_OccupationRate_lag_1,28005_OccupationRate_lag_7,28005_OccupationRate_lag_30,28008_OccupationRate_lag_1,28008_OccupationRate_lag_7,28008_OccupationRate_lag_30,28014_OccupationRate_lag_1,28014_OccupationRate_lag_7,28014_OccupationRate_lag_30,28012_OccupationRate_lag_1,28012_OccupationRate_lag_7,28012_OccupationRate_lag_30,28009_OccupationRate_lag_1,28009_OccupationRate_lag_7,28009_OccupationRate_lag_30,28007_OccupationRate_lag_1,28007_OccupationRate_lag_7,28007_OccupationRate_lag_30,28001_OccupationRate_lag_1,28001_OccupationRate_lag_7,28001_OccupationRate_lag_30,28006_OccupationRate_lag_1,28006_OccupationRate_lag_7,28006_OccupationRate_lag_30,28020_OccupationRate_lag_1,28020_OccupationRate_lag_7,28020_OccupationRate_lag_30,28002_OccupationRate_lag_1,28002_OccupationRate_lag_7,28002_OccupationRate_lag_30,28045_OccupationRate_lag_1,28045_OccupationRate_lag_7,28045_OccupationRate_lag_30,28003_OccupationRate_lag_1,28003_OccupationRate_lag_7,28003_OccupationRate_lag_30,28046_OccupationRate_lag_1,28046_OccupationRate_lag_7,28046_OccupationRate_lag_30,28036_OccupationRate_lag_1,28036_OccupationRate_lag_7,28036_OccupationRate_lag_30,28016_OccupationRate_lag_1,28016_OccupationRate_lag_7,28016_OccupationRate_lag_30
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-27,0.299477,0.385396,0.162701,0.203048,0.494028,0.653363,0.407425,0.550138,0.632122,0.667118,0.347915,0.643278,0.526451,0.603318,0.660714,0.490738,0.710355,0.537703,0.715044,27,5,-0.781831,0.623490,0.340093,0.307210,0.600279,0.498247,0.420676,0.599477,0.302904,0.339709,0.476803,0.328088,0.425362,0.463109,0.584268,0.444961,0.528416,0.587575,0.551885,0.464310,0.461263,0.508024,0.497831,0.578990,0.685509,0.526642,0.550423,0.467378,0.461239,0.697448,0.494968,0.523615,0.332013,0.386173,0.400570,0.525242,0.487761,0.408267,0.440820,0.430865,0.412507,0.466641,0.457499,0.379203,0.622214,0.479264,0.503945,0.504513,0.530644,0.482832,0.514166,0.569969,0.426700,0.420828,0.421820,0.412255,0.633028,0.735652,0.575045
2019-01-28,0.368841,0.332530,0.215184,0.382485,0.444240,0.560981,0.454824,0.507675,0.597810,0.568873,0.498269,0.551532,0.519191,0.628977,0.582752,0.668240,0.658488,0.633393,0.870567,28,4,0.000000,1.000000,0.299477,0.378196,0.469131,0.385396,0.383934,0.647406,0.162701,0.295953,0.373213,0.203048,0.478980,0.495347,0.494028,0.476855,0.499397,0.653363,0.469613,0.471724,0.407425,0.426249,0.427057,0.550138,0.533291,0.577803,0.632122,0.478776,0.498315,0.667118,0.430868,0.417687,0.347915,0.511077,0.493282,0.643278,0.490783,0.506307,0.526451,0.474248,0.432461,0.603318,0.466239,0.450076,0.660714,0.593569,0.496686,0.490738,0.473800,0.538744,0.710355,0.624053,0.471218,0.537703,0.585181,0.488663,0.715044,0.510563,0.395797
2019-01-29,0.448655,0.436226,0.428039,0.453835,0.521565,0.520196,0.451750,0.458112,0.482685,0.652618,0.372763,0.426589,0.468322,0.489508,0.538020,0.620786,0.532898,0.645218,0.799283,29,3,0.781831,0.623490,0.368841,0.374714,0.509503,0.332530,0.420573,0.538589,0.215184,0.459331,0.341918,0.382485,0.405787,0.418613,0.444240,0.526350,0.540862,0.560981,0.461794,0.446960,0.454824,0.439801,0.443197,0.507675,0.483143,0.612435,0.597810,0.486050,0.521352,0.568873,0.646168,0.489750,0

In [29]:
train.fillna(0, inplace=True)
train.head(31)

C:\Users\Aeroengy\Anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days,sin_weekday,cos_weekday,28013_OccupationRate_lag_1,28013_OccupationRate_lag_7,28013_OccupationRate_lag_30,28015_OccupationRate_lag_1,28015_OccupationRate_lag_7,28015_OccupationRate_lag_30,28004_OccupationRate_lag_1,28004_OccupationRate_lag_7,28004_OccupationRate_lag_30,28010_OccupationRate_lag_1,28010_OccupationRate_lag_7,28010_OccupationRate_lag_30,28005_OccupationRate_lag_1,28005_OccupationRate_lag_7,28005_OccupationRate_lag_30,28008_OccupationRate_lag_1,28008_OccupationRate_lag_7,28008_OccupationRate_lag_30,28014_OccupationRate_lag_1,28014_OccupationRate_lag_7,28014_OccupationRate_lag_30,28012_OccupationRate_lag_1,28012_OccupationRate_lag_7,28012_OccupationRate_lag_30,28009_OccupationRate_lag_1,28009_OccupationRate_lag_7,28009_OccupationRate_lag_30,28007_OccupationRate_lag_1,28007_OccupationRate_lag_7,28007_OccupationRate_lag_30,28001_OccupationRate_lag_1,28001_OccupationRate_lag_7,28001_OccupationRate_lag_30,28006_OccupationRate_lag_1,28006_OccupationRate_lag_7,28006_OccupationRate_lag_30,28020_OccupationRate_lag_1,28020_OccupationRate_lag_7,28020_OccupationRate_lag_30,28002_OccupationRate_lag_1,28002_OccupationRate_lag_7,28002_OccupationRate_lag_30,28045_OccupationRate_lag_1,28045_OccupationRate_lag_7,28045_OccupationRate_lag_30,28003_OccupationRate_lag_1,28003_OccupationRate_lag_7,28003_OccupationRate_lag_30,28046_OccupationRate_lag_1,28046_OccupationRate_lag_7,28046_OccupationRate_lag_30,28036_OccupationRate_lag_1,28036_OccupationRate_lag_7,28036_OccupationRate_lag_30,28016_OccupationRate_lag_1,28016_OccupationRate_lag_7,28016_OccupationRate_lag_30
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-08-01,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932,1,184,0.974928,-0.222521,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-08-02,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505,2,183,0.433884,-0.900969,0.222725,0.000000,0.000000,0.357760,0.000000,0.00000,0.223711,0.000000,0.000000,0.526093,0.000000,0.000000,0.508977,0.000000,0.000000,0.382280,0.000000,0.00000,0.376431,0.000000,0.000000,0.525494,0.000000,0.000000,0.459238,0.000000,0.000000,0.637176,0.000000,0.000000,0.374077,0.000000,0.000000,0.416704,0.000000,0.000000,0.558309,0.000000,0.000000,0.560564,0.000000,0.000000,0.497305,0.000000,0.000000,0.528046,0.000000,0.000000,0.392157,0.000000,0.000000,0.588095,0.000000,0.000000,0.256932,0.000000,0.000000
2018-08-03,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307,3,182,-0.433884,-0.900969,0.269197,0.000000,0.000000,0.309609,0.000000,0.00000,0.287564,0.000000,0.000000,0.429735,0.000000,0.000000,0.494181,0.000000,0.000000,0.415399,0.000000,0.00000,0.429609,0.000000,0.000000,0.525772,0.000000,0.000000,0.472398,0.000000,0.000000,0.586897,0.000000,0.000000,0

In [30]:
def bucle_lag_function(df):
    columns = [x for x in df if 'OccupationRate' in x]
    for column in columns:
        df = lag_function(df, column)
    df.fillna(0, inplace=True)
    return df

## 4. Weather feature engineering

In [31]:
df_weather = pd.read_json(os.path.join(data_path, 'METEO.txt'))
df_weather.head()

,altitud,dir,fecha,horaPresMax,horaPresMin,horaracha,horatmax,horatmin,indicativo,nombre,prec,presMax,presMin,provincia,racha,tmax,tmed,tmin,velmedia
0,667,22.0,2018-07-29,24,05,11:30,14:10,16:30,3195,"MADRID, RETIRO","0,0","939,9","936,9",MADRID,"7,5","32,6","26,2","19,8","1,7"
1,667,28.0,2018-07-30,09,18,15:00,14:40,05:30,3195,"MADRID, RETIRO","0,0","941,2","938,7",MADRID,"6,9","34,1","26,8","19,4","1,4"
2,667,16.0,2018-07-31,08,18,15:00,14:10,04:50,3195,"MADRID, RETIRO","0,0","941,8","938,7",MADRID,"6,4","35,3","28,2","21,1","1,1"
3,667,4.0,2018-08-01,10,18,00:40,13:50,05:45,3195,"MADRID, RETIRO","0,0","941,1","938,8",MADRID,"6,4","37,0","30,2","23,3","0,6"
4,667,7.0,2018-08-02,08,18,22:50,14:10,06:10,3195,"MADRID, RETIRO","0,0","943,8","939,7",MADRID,"7,5","38,5","31,6","24,6","1,1"


In [32]:
df_weather['fecha'] = pd.to_datetime(df_weather['fecha'], format='%Y-%m-%d')
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 19 columns):
altitud        196 non-null int64
dir            194 non-null float64
fecha          196 non-null datetime64[ns]
horaPresMax    196 non-null object
horaPresMin    196 non-null object
horaracha      194 non-null object
horatmax       196 non-null object
horatmin       196 non-null object
indicativo     196 non-null int64
nombre         196 non-null object
prec           196 non-null object
presMax        196 non-null object
presMin        196 non-null object
provincia      196 non-null object
racha          194 non-null object
tmax           196 non-null object
tmed           196 non-null object
tmin           196 non-null object
velmedia       195 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(15)
memory usage: 29.2+ KB


In [33]:
selected_columns = ['fecha', 'prec', 'presMax', 'tmed', 'velmedia']

In [34]:
df_weather_selected = df_weather[selected_columns]
df_weather_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
fecha       196 non-null datetime64[ns]
prec        196 non-null object
presMax     196 non-null object
tmed        196 non-null object
velmedia    195 non-null object
dtypes: datetime64[ns](1), object(4)
memory usage: 7.7+ KB


In [35]:
def convert_to_number(x):
    try:
        x = x.replace(',','.')
    except:
        x = 0
    try:
        x = float(x)
    except:
        x = 0
    return x

In [36]:
for column in [x for x in selected_columns if 'fecha' not in x]:
    print(column)
    df_weather_selected[column] = df_weather_selected[column].apply(convert_to_number)

prec
presMax
tmed
velmedia


C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
df_weather_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
fecha       196 non-null datetime64[ns]
prec        196 non-null float64
presMax     196 non-null float64
tmed        196 non-null float64
velmedia    196 non-null float64
dtypes: datetime64[ns](1), float64(4)
memory usage: 7.7 KB


In [38]:
train = train.merge(df_weather_selected[['fecha', 'prec', 'presMax', 'tmed', 'velmedia']], how='left', left_index=True, right_on='fecha')
train.drop('fecha', axis=1, inplace=True)
train.reset_index(drop=True, inplace=True)
train.head()

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days,sin_weekday,cos_weekday,28013_OccupationRate_lag_1,28013_OccupationRate_lag_7,28013_OccupationRate_lag_30,28015_OccupationRate_lag_1,28015_OccupationRate_lag_7,28015_OccupationRate_lag_30,28004_OccupationRate_lag_1,28004_OccupationRate_lag_7,28004_OccupationRate_lag_30,28010_OccupationRate_lag_1,28010_OccupationRate_lag_7,28010_OccupationRate_lag_30,28005_OccupationRate_lag_1,28005_OccupationRate_lag_7,28005_OccupationRate_lag_30,28008_OccupationRate_lag_1,28008_OccupationRate_lag_7,28008_OccupationRate_lag_30,28014_OccupationRate_lag_1,28014_OccupationRate_lag_7,28014_OccupationRate_lag_30,28012_OccupationRate_lag_1,28012_OccupationRate_lag_7,28012_OccupationRate_lag_30,28009_OccupationRate_lag_1,28009_OccupationRate_lag_7,28009_OccupationRate_lag_30,28007_OccupationRate_lag_1,28007_OccupationRate_lag_7,28007_OccupationRate_lag_30,28001_OccupationRate_lag_1,28001_OccupationRate_lag_7,28001_OccupationRate_lag_30,28006_OccupationRate_lag_1,28006_OccupationRate_lag_7,28006_OccupationRate_lag_30,28020_OccupationRate_lag_1,28020_OccupationRate_lag_7,28020_OccupationRate_lag_30,28002_OccupationRate_lag_1,28002_OccupationRate_lag_7,28002_OccupationRate_lag_30,28045_OccupationRate_lag_1,28045_OccupationRate_lag_7,28045_OccupationRate_lag_30,28003_OccupationRate_lag_1,28003_OccupationRate_lag_7,28003_OccupationRate_lag_30,28046_OccupationRate_lag_1,28046_OccupationRate_lag_7,28046_OccupationRate_lag_30,28036_OccupationRate_lag_1,28036_OccupationRate_lag_7,28036_OccupationRate_lag_30,28016_OccupationRate_lag_1,28016_OccupationRate_lag_7,28016_OccupationRate_lag_30,prec,presMax,tmed,velmedia
0,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932,1,184,0.974928,-0.222521,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,941.1,30.2,0.6
1,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505,2,183,0.433884,-0.900969,0.222725,0.0,0.0,0.357760,0.0,0.0,0.223711,0.0,0.0,0.526093,0.0,0.0,0.508977,0.0,0.0,0.382280,0.0,0.0,0.376431,0.0,0.0,0.525494,0.0,0.0,0.459238,0.0,0.0,0.637176,0.0,0.0,0.374077,0.0,0.0,0.416704,0.0,0.0,0.558309,0.0,0.0,0.560564,0.0,0.0,0.497305,0.0,0.0,0.528046,0.0,0.0,0.392157,0.0,0.0,0.588095,0.0,0.0,0.256932,0.0,0.0,0.0,943.8,31.6,1.1
2,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307,3,182,-0.433884,-0.900969,0.269197,0.0,0.0,0.309609,0.0,0.0,0.287564,0.0,0.0,0.429735,0.0,0.0,0.494181,0.0,0.0,0.415399,0.0,0.0,0.429609,0.0,0.0,0.525772,0.0,0.0,0.472398,0.0,0.0,0.586897,0.0,0.0,0.403961,0.0,0.0,0.342030,0.0,0.0,0.498043,0.0,0.0,0.441732,0.0,0.0,0.486419,0.0,0.0,0.483920,0.0,0.0,0.494270,0.0,0.0,0.634788,0.0,0.0,0.292505,0.0,0.0,0.0,942.4,32.9,1.7
3,0.365708,0.366367,0.181165,0.393908,0.389521,0.472140,0.501181,0.456476,0.515255,0.554839,0.512104,0.445342,0.493756,0.476997,0.544531,0.520857,0.519066,0.555746,0.491611,4,181,-0.974928,-0.222521,0.312698,0.0,0.0,0.328358,0.0,0.0,0.246035,0.0,0.0,0.376402,0.0,0.0,0.470126,0.0,0.0,0.440479,0.0,0.0,0.544573,0.0,0.0,0.528967,0.0,0.0,0.407599,0.0,0.

In [39]:
train[['prec', 'presMax', 'tmed', 'velmedia']].fillna(train[['prec', 'presMax', 'tmed', 'velmedia']].mean(), inplace=True)
train[['prec', 'presMax', 'tmed', 'velmedia']]

C:\Users\Aeroengy\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,prec,presMax,tmed,velmedia
0,0.0,941.1,30.2,0.6
1,0.0,943.8,31.6,1.1
2,0.0,942.4,32.9,1.7
3,0.0,943.3,31.0,0.0
4,0.0,942.1,31.0,0.8
5,0.0,941.0,30.6,1.7
6,0.0,937.7,30.7,2.8
7,0.0,937.2,27.9,2.8
8,0.0,943.0,25.4,2.2
9,0.0,945.8,24.4,2.2


In [40]:
train

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days,sin_weekday,cos_weekday,28013_OccupationRate_lag_1,28013_OccupationRate_lag_7,28013_OccupationRate_lag_30,28015_OccupationRate_lag_1,28015_OccupationRate_lag_7,28015_OccupationRate_lag_30,28004_OccupationRate_lag_1,28004_OccupationRate_lag_7,28004_OccupationRate_lag_30,28010_OccupationRate_lag_1,28010_OccupationRate_lag_7,28010_OccupationRate_lag_30,28005_OccupationRate_lag_1,28005_OccupationRate_lag_7,28005_OccupationRate_lag_30,28008_OccupationRate_lag_1,28008_OccupationRate_lag_7,28008_OccupationRate_lag_30,28014_OccupationRate_lag_1,28014_OccupationRate_lag_7,28014_OccupationRate_lag_30,28012_OccupationRate_lag_1,28012_OccupationRate_lag_7,28012_OccupationRate_lag_30,28009_OccupationRate_lag_1,28009_OccupationRate_lag_7,28009_OccupationRate_lag_30,28007_OccupationRate_lag_1,28007_OccupationRate_lag_7,28007_OccupationRate_lag_30,28001_OccupationRate_lag_1,28001_OccupationRate_lag_7,28001_OccupationRate_lag_30,28006_OccupationRate_lag_1,28006_OccupationRate_lag_7,28006_OccupationRate_lag_30,28020_OccupationRate_lag_1,28020_OccupationRate_lag_7,28020_OccupationRate_lag_30,28002_OccupationRate_lag_1,28002_OccupationRate_lag_7,28002_OccupationRate_lag_30,28045_OccupationRate_lag_1,28045_OccupationRate_lag_7,28045_OccupationRate_lag_30,28003_OccupationRate_lag_1,28003_OccupationRate_lag_7,28003_OccupationRate_lag_30,28046_OccupationRate_lag_1,28046_OccupationRate_lag_7,28046_OccupationRate_lag_30,28036_OccupationRate_lag_1,28036_OccupationRate_lag_7,28036_OccupationRate_lag_30,28016_OccupationRate_lag_1,28016_OccupationRate_lag_7,28016_OccupationRate_lag_30,prec,presMax,tmed,velmedia
0,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932,1,184,0.974928,-0.222521,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,941.1,30.2,0.6
1,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505,2,183,0.433884,-0.900969,0.222725,0.000000,0.000000,0.357760,0.000000,0.000000,0.223711,0.000000,0.000000,0.526093,0.000000,0.000000,0.508977,0.000000,0.000000,0.382280,0.000000,0.000000,0.376431,0.000000,0.000000,0.525494,0.000000,0.000000,0.459238,0.000000,0.000000,0.637176,0.000000,0.000000,0.374077,0.000000,0.000000,0.416704,0.000000,0.000000,0.558309,0.000000,0.000000,0.560564,0.000000,0.000000,0.497305,0.000000,0.000000,0.528046,0.000000,0.000000,0.392157,0.000000,0.000000,0.588095,0.000000,0.000000,0.256932,0.000000,0.000000,0.0,943.8,31.6,1.1
2,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307,3,182,-0.433884,-0.900969,0.269197,0.000000,0.000000,0.309609,0.000000,0.000000,0.287564,0.000000,0.000000,0.429735,0.000000,0.000000,0.494181,0.000000,0.000000,0.415399,0.000000,0.000000,0.429609,0.000000,0.000000,0.525772,0.000000,0.000000,0.472398,0.000000,0.000000,0.586897,0.000000,0.000000,0.403961,0.000000,0.000000,0.342030,0.0000

In [57]:
def make_weather_features(df, df_weather):
    df_weather['fecha'] = pd.to_datetime(df_weather['fecha'], format='%Y-%m-%d')
    selected_columns = ['fecha', 'prec', 'presMax', 'tmed', 'velmedia']
    df_weather_selected = df_weather[selected_columns]
    for column in [x for x in selected_columns if 'fecha' not in x]:
        df_weather_selected[column] = df_weather_selected[column].apply(convert_to_number)
    df = df.merge(df_weather_selected[['fecha', 'prec', 'presMax', 'tmed', 'velmedia']], how='left', left_index=True, right_on='fecha')
    df.drop('fecha', axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df[['prec', 'presMax', 'tmed', 'velmedia']].fillna(df[['prec', 'presMax', 'tmed', 'velmedia']].mean(), inplace=True)
    return df

## 5. The function for all the feature engineering.

In [42]:
def feature_engineering(df, df_weather):
    df = make_time_features(df)
    df = bucle_lag_function(df)
    df = make_weather_features(df, df_weather)
    return df

In [43]:
df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=False )
df_total_bases = pre.total_bases_dataset(data_path)
df_occupation_rate = pre.make_df_occupation_rate(df_dock_bikes, df_total_bases)
df_test_fun_feature_engineering = df_occupation_rate[:-1]
df_test_fun_feature_engineering.head()

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate
Date,,,,,,,,,,,,,,,,,,,
2018-08-01,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932
2018-08-02,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505
2018-08-03,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307
2018-08-04,0.365708,0.366367,0.181165,0.393908,0.389521,0.472140,0.501181,0.456476,0.515255,0.554839,0.512104,0.445342,0.493756,0.476997,0.544531,0.520857,0.519066,0.555746,0.491611
2018-08-05,0.352376,0.460019,0.368582,0.520932,0.378999,0.521230,0.407549,0.415905,0.442334,0.512357,0.470336,0.532366,0.468555,0.500566,0.485523,0.548259,0.509181,0.499412,0.452080


In [44]:
df_weather_test_fun_feature_engineering = pd.read_json(os.path.join(data_path, 'METEO.txt'))
df_weather_test_fun_feature_engineering.head(1)

,altitud,dir,fecha,horaPresMax,horaPresMin,horaracha,horatmax,horatmin,indicativo,nombre,prec,presMax,presMin,provincia,racha,tmax,tmed,tmin,velmedia
0,667,22.0,2018-07-29,24,05,11:30,14:10,16:30,3195,"MADRID, RETIRO","0,0","939,9","936,9",MADRID,"7,5","32,6","26,2","19,8","1,7"


In [45]:
train_test_fun_feature_engineering = feature_engineering(df_test_fun_feature_engineering, df_weather_test_fun_feature_engineering)

C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users

In [46]:
train_test_fun_feature_engineering.head()

,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days,sin_weekday,cos_weekday,28013_OccupationRate_lag_1,28013_OccupationRate_lag_7,28013_OccupationRate_lag_30,28015_OccupationRate_lag_1,28015_OccupationRate_lag_7,28015_OccupationRate_lag_30,28004_OccupationRate_lag_1,28004_OccupationRate_lag_7,28004_OccupationRate_lag_30,28010_OccupationRate_lag_1,28010_OccupationRate_lag_7,28010_OccupationRate_lag_30,28005_OccupationRate_lag_1,28005_OccupationRate_lag_7,28005_OccupationRate_lag_30,28008_OccupationRate_lag_1,28008_OccupationRate_lag_7,28008_OccupationRate_lag_30,28014_OccupationRate_lag_1,28014_OccupationRate_lag_7,28014_OccupationRate_lag_30,28012_OccupationRate_lag_1,28012_OccupationRate_lag_7,28012_OccupationRate_lag_30,28009_OccupationRate_lag_1,28009_OccupationRate_lag_7,28009_OccupationRate_lag_30,28007_OccupationRate_lag_1,28007_OccupationRate_lag_7,28007_OccupationRate_lag_30,28001_OccupationRate_lag_1,28001_OccupationRate_lag_7,28001_OccupationRate_lag_30,28006_OccupationRate_lag_1,28006_OccupationRate_lag_7,28006_OccupationRate_lag_30,28020_OccupationRate_lag_1,28020_OccupationRate_lag_7,28020_OccupationRate_lag_30,28002_OccupationRate_lag_1,28002_OccupationRate_lag_7,28002_OccupationRate_lag_30,28045_OccupationRate_lag_1,28045_OccupationRate_lag_7,28045_OccupationRate_lag_30,28003_OccupationRate_lag_1,28003_OccupationRate_lag_7,28003_OccupationRate_lag_30,28046_OccupationRate_lag_1,28046_OccupationRate_lag_7,28046_OccupationRate_lag_30,28036_OccupationRate_lag_1,28036_OccupationRate_lag_7,28036_OccupationRate_lag_30,28016_OccupationRate_lag_1,28016_OccupationRate_lag_7,28016_OccupationRate_lag_30,prec,presMax,tmed,velmedia
3,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932,1,184,0.974928,-0.222521,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,941.1,30.2,0.6
4,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505,2,183,0.433884,-0.900969,0.222725,0.0,0.0,0.357760,0.0,0.0,0.223711,0.0,0.0,0.526093,0.0,0.0,0.508977,0.0,0.0,0.382280,0.0,0.0,0.376431,0.0,0.0,0.525494,0.0,0.0,0.459238,0.0,0.0,0.637176,0.0,0.0,0.374077,0.0,0.0,0.416704,0.0,0.0,0.558309,0.0,0.0,0.560564,0.0,0.0,0.497305,0.0,0.0,0.528046,0.0,0.0,0.392157,0.0,0.0,0.588095,0.0,0.0,0.256932,0.0,0.0,0.0,943.8,31.6,1.1
5,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307,3,182,-0.433884,-0.900969,0.269197,0.0,0.0,0.309609,0.0,0.0,0.287564,0.0,0.0,0.429735,0.0,0.0,0.494181,0.0,0.0,0.415399,0.0,0.0,0.429609,0.0,0.0,0.525772,0.0,0.0,0.472398,0.0,0.0,0.586897,0.0,0.0,0.403961,0.0,0.0,0.342030,0.0,0.0,0.498043,0.0,0.0,0.441732,0.0,0.0,0.486419,0.0,0.0,0.483920,0.0,0.0,0.494270,0.0,0.0,0.634788,0.0,0.0,0.292505,0.0,0.0,0.0,942.4,32.9,1.7
6,0.365708,0.366367,0.181165,0.393908,0.389521,0.472140,0.501181,0.456476,0.515255,0.554839,0.512104,0.445342,0.493756,0.476997,0.544531,0.520857,0.519066,0.555746,0.491611,4,181,-0.974928,-0.222521,0.312698,0.0,0.0,0.328358,0.0,0.0,0.246035,0.0,0.0,0.376402,0.0,0.0,0.470126,0.0,0.0,0.440479,0.0,0.0,0.544573,0.0,0.0,0.528967,0.0,0.0,0.407599,0.0,0.

In [47]:
np.sum(train_test_fun_feature_engineering.values - train.values)

0.0

## 6. Model

### A. First Model

In [48]:
columns = train.columns
X_columns = [x for x in columns if '28' in x and 'lag' in x]
X_columns = X_columns + ['prec', 'presMax', 'tmed', 'velmedia', 'day_month', 'back_days', 'sin_weekday', 'cos_weekday']
X_columns

['28013_OccupationRate_lag_1',
 '28013_OccupationRate_lag_7',
 '28013_OccupationRate_lag_30',
 '28015_OccupationRate_lag_1',
 '28015_OccupationRate_lag_7',
 '28015_OccupationRate_lag_30',
 '28004_OccupationRate_lag_1',
 '28004_OccupationRate_lag_7',
 '28004_OccupationRate_lag_30',
 '28010_OccupationRate_lag_1',
 '28010_OccupationRate_lag_7',
 '28010_OccupationRate_lag_30',
 '28005_OccupationRate_lag_1',
 '28005_OccupationRate_lag_7',
 '28005_OccupationRate_lag_30',
 '28008_OccupationRate_lag_1',
 '28008_OccupationRate_lag_7',
 '28008_OccupationRate_lag_30',
 '28014_OccupationRate_lag_1',
 '28014_OccupationRate_lag_7',
 '28014_OccupationRate_lag_30',
 '28012_OccupationRate_lag_1',
 '28012_OccupationRate_lag_7',
 '28012_OccupationRate_lag_30',
 '28009_OccupationRate_lag_1',
 '28009_OccupationRate_lag_7',
 '28009_OccupationRate_lag_30',
 '28007_OccupationRate_lag_1',
 '28007_OccupationRate_lag_7',
 '28007_OccupationRate_lag_30',
 '28001_OccupationRate_lag_1',
 '28001_OccupationRate_lag_7'

In [49]:
y_columns = [y for y in columns if '28' in y and 'lag' not in y]
y_columns

['28013_OccupationRate',
 '28015_OccupationRate',
 '28004_OccupationRate',
 '28010_OccupationRate',
 '28005_OccupationRate',
 '28008_OccupationRate',
 '28014_OccupationRate',
 '28012_OccupationRate',
 '28009_OccupationRate',
 '28007_OccupationRate',
 '28001_OccupationRate',
 '28006_OccupationRate',
 '28020_OccupationRate',
 '28002_OccupationRate',
 '28045_OccupationRate',
 '28003_OccupationRate',
 '28046_OccupationRate',
 '28036_OccupationRate',
 '28016_OccupationRate']

In [62]:
X = train.iloc[:-1][X_columns]
y = train.iloc[:-1][y_columns]

In [63]:
print(X.shape)
print(y.shape)

(183, 65)
(183, 19)


In [67]:
def get_X_y(df):
    columns = df.columns
    X_columns = [x for x in columns if '28' in x and 'lag' in x]
    X_columns = X_columns + ['prec', 'presMax', 'tmed', 'velmedia', 'day_month', 'back_days', 'sin_weekday', 'cos_weekday']
    y_columns = [y for y in columns if '28' in y and 'lag' not in y]
    X = df[X_columns]
    y = df[y_columns]
    return (X, y)

In [64]:
from sklearn.ensemble import RandomForestRegressor

In [65]:
model = RandomForestRegressor(max_depth=4, n_estimators=50, random_state=20)

In [66]:
model.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
           oob_score=False, random_state=20, verbose=0, warm_start=False)

In [68]:
X_validation = train.iloc[-1:][X_columns]
y_real = train.iloc[-1:][Y_columns]

In [69]:
print(X_validation.shape)
print(y_real.shape)

(1, 65)
(1, 19)


In [70]:
y_predicted_validation = model.predict(X_validation)
y_predicted_validation

array([[0.48410817, 0.50396695, 0.41894719, 0.45006248, 0.52778624,
        0.48235181, 0.42537803, 0.55672215, 0.48426719, 0.55580409,
        0.37294202, 0.41252537, 0.44997779, 0.4373378 , 0.52249467,
        0.51205203, 0.44831712, 0.49480516, 0.49649768]])

In [71]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt

In [72]:
def metrics(y_real, y_predicted):
    mae = mean_absolute_error(y_real, y_predicted)
    print('MAE: %f' % mae)
    mse = mean_squared_error(y_real, y_predicted)
    print('MSE: %f' % mse)
    rmse = sqrt(mse)
    print('RMSE: %f' % rmse)
    
    return mae, mse, rmse

In [73]:
metrics(y_real, y_predicted_validation)

MAE: 0.065029
MSE: 0.005279
RMSE: 0.072658


(0.06502930767230165, 0.005279217395703882, 0.07265822318020089)

### B. RandomizedSearchCV Model

In [74]:
from sklearn.model_selection import RandomizedSearchCV

In [101]:
RandomizedModel = RandomizedSearchCV(RandomForestRegressor(),
                                    param_distributions={'n_estimators' : [10,30,50,80,100,130,150],
                                                        'max_depth' : [5,10,15,20,25,30,35,40]},
                                    scoring='neg_mean_squared_error',
                                    n_iter=150, n_jobs=-1, verbose=2)

In [102]:
X_RandSearch, y_RandSearch = get_X_y(train)
print(X_RandSearch.shape, y_RandSearch.shape)

(184, 65) (184, 19)


In [103]:
RandomizedModel.fit(X_RandSearch, y_RandSearch)

C:\Users\Aeroengy\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Aeroengy\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 56 is smaller than n_iter=150. Running 56 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 56 candidates, totalling 168 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 168 out of 168 | elapsed:   24.7s finished


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=150, n_jobs=-1,
          param_distributions={'n_estimators': [10, 30, 50, 80, 100, 130, 150], 'max_depth': [5, 10, 15, 20, 25, 30, 35, 40]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=2)

In [104]:
RandomizedModel.best_params_

{'n_estimators': 50, 'max_depth': 40}

In [105]:
RandomizedModel.best_score_

-0.006187442769413419

In [106]:
model_hiper = RandomizedModel.best_estimator_

### C. Test

In [107]:
df_dock_bikes = pre.make_all_dataset(data_path, 'dock_bikes',by_postal_code=True, verbose=False )
df_total_bases = pre.total_bases_dataset(data_path)
df_occupation_rate = pre.make_df_occupation_rate(df_dock_bikes, df_total_bases)
df_weather = pd.read_json(os.path.join(data_path, 'METEO.txt'))
df_validation = feature_engineering(df_occupation_rate, df_weather)
print(df_validation.shape)
df_validation.head(3)

C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  
C:\Users\Aeroengy\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(185, 84)


C:\Users\Aeroengy\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,28013_OccupationRate,28015_OccupationRate,28004_OccupationRate,28010_OccupationRate,28005_OccupationRate,28008_OccupationRate,28014_OccupationRate,28012_OccupationRate,28009_OccupationRate,28007_OccupationRate,28001_OccupationRate,28006_OccupationRate,28020_OccupationRate,28002_OccupationRate,28045_OccupationRate,28003_OccupationRate,28046_OccupationRate,28036_OccupationRate,28016_OccupationRate,day_month,back_days,sin_weekday,cos_weekday,28013_OccupationRate_lag_1,28013_OccupationRate_lag_7,28013_OccupationRate_lag_30,28015_OccupationRate_lag_1,28015_OccupationRate_lag_7,28015_OccupationRate_lag_30,28004_OccupationRate_lag_1,28004_OccupationRate_lag_7,28004_OccupationRate_lag_30,28010_OccupationRate_lag_1,28010_OccupationRate_lag_7,28010_OccupationRate_lag_30,28005_OccupationRate_lag_1,28005_OccupationRate_lag_7,28005_OccupationRate_lag_30,28008_OccupationRate_lag_1,28008_OccupationRate_lag_7,28008_OccupationRate_lag_30,28014_OccupationRate_lag_1,28014_OccupationRate_lag_7,28014_OccupationRate_lag_30,28012_OccupationRate_lag_1,28012_OccupationRate_lag_7,28012_OccupationRate_lag_30,28009_OccupationRate_lag_1,28009_OccupationRate_lag_7,28009_OccupationRate_lag_30,28007_OccupationRate_lag_1,28007_OccupationRate_lag_7,28007_OccupationRate_lag_30,28001_OccupationRate_lag_1,28001_OccupationRate_lag_7,28001_OccupationRate_lag_30,28006_OccupationRate_lag_1,28006_OccupationRate_lag_7,28006_OccupationRate_lag_30,28020_OccupationRate_lag_1,28020_OccupationRate_lag_7,28020_OccupationRate_lag_30,28002_OccupationRate_lag_1,28002_OccupationRate_lag_7,28002_OccupationRate_lag_30,28045_OccupationRate_lag_1,28045_OccupationRate_lag_7,28045_OccupationRate_lag_30,28003_OccupationRate_lag_1,28003_OccupationRate_lag_7,28003_OccupationRate_lag_30,28046_OccupationRate_lag_1,28046_OccupationRate_lag_7,28046_OccupationRate_lag_30,28036_OccupationRate_lag_1,28036_OccupationRate_lag_7,28036_OccupationRate_lag_30,28016_OccupationRate_lag_1,28016_OccupationRate_lag_7,28016_OccupationRate_lag_30,prec,presMax,tmed,velmedia
0,0.222725,0.357760,0.223711,0.526093,0.508977,0.382280,0.376431,0.525494,0.459238,0.637176,0.374077,0.416704,0.558309,0.560564,0.497305,0.528046,0.392157,0.588095,0.256932,1,185,0.974928,-0.222521,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,941.1,30.2,0.6
1,0.269197,0.309609,0.287564,0.429735,0.494181,0.415399,0.429609,0.525772,0.472398,0.586897,0.403961,0.342030,0.498043,0.441732,0.486419,0.483920,0.494270,0.634788,0.292505,2,184,0.433884,-0.900969,0.222725,0.0,0.0,0.357760,0.0,0.0,0.223711,0.0,0.0,0.526093,0.0,0.0,0.508977,0.0,0.0,0.382280,0.0,0.0,0.376431,0.0,0.0,0.525494,0.0,0.0,0.459238,0.0,0.0,0.637176,0.0,0.0,0.374077,0.0,0.0,0.416704,0.0,0.0,0.558309,0.0,0.0,0.560564,0.0,0.0,0.497305,0.0,0.0,0.528046,0.0,0.0,0.392157,0.0,0.0,0.588095,0.0,0.0,0.256932,0.0,0.0,0.0,943.8,31.6,1.1
2,0.312698,0.328358,0.246035,0.376402,0.470126,0.440479,0.544573,0.528967,0.407599,0.594210,0.426108,0.401211,0.472252,0.569466,0.577809,0.409132,0.453639,0.669497,0.453307,3,183,-0.433884,-0.900969,0.269197,0.0,0.0,0.309609,0.0,0.0,0.287564,0.0,0.0,0.429735,0.0,0.0,0.494181,0.0,0.0,0.415399,0.0,0.0,0.429609,0.0,0.0,0.525772,0.0,0.0,0.472398,0.0,0.0,0.586897,0.0,0.0,0.403961,0.0,0.0,0.342030,0.0,0.0,0.498043,0.0,0.0,0.441732,0.0,0.0,0.486419,0.0,0.0,0.483920,0.0,0.0,0.494270,0.0,0.0,0.634788,0.0,0.0,0.292505,0.0,0.0,0.0,942.4,32.9,1.7


In [97]:
X_validation, y_val_real = get_X_y(df_validation.iloc[-1:])

In [98]:
print(X_validation.shape)
print(y_val_real.shape)

(1, 65)
(1, 19)


In [99]:
y_val_predicted = model_hiper.predict(X_validation)

In [100]:
metrics(y_val_real, y_val_predicted)

MAE: 0.101551
MSE: 0.014033
RMSE: 0.118462


(0.10155121832583776, 0.014033289135718335, 0.11846218441223484)